In [69]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [70]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [71]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [72]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [73]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [74]:
# View all of the classes that automap found

#HELP: WHEN I VIEWED THE CLASSES, IT RETURNED AN EMPTY LIST
# I can see using the SQLiteExplorer that there are two tables that I should be picking up.
#1) measurement and 2)station
Base.classes.keys()

['measurement', 'station']

In [75]:
# Save references to each table


In [76]:
# Create our session (link) from Python to the DB


# Exploratory Precipitation Analysis

In [77]:
# Find the most recent date in the data set.

In [78]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [79]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [80]:
# Design a query to calculate the total number stations in the dataset


In [81]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [82]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [83]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [84]:
# Close Session
session.close()

NameError: name 'session' is not defined